<a href="https://colab.research.google.com/github/mohammadsjahanbakhsh/deep_learning_keras/blob/main/deep_practise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

In [ ]:
! kaggle datasets download -d jangedoo/utkface-new

In [ ]:
!unzip /content/utkface-new.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import load_img
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D,Dropout,Flatten,MaxPooling2D,Input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from tqdm.notebook import tqdm
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

In [ ]:
# primary_directory = '/content/UTKFace'
primary_directory = '/content/crop_part1'



In [ ]:

age_labels = []
gender_labels = []
image_paths = []

image_filenames = os.listdir(primary_directory)
random.shuffle(image_filenames)

for image in tqdm(image_filenames):

  image_path = os.path.join(primary_directory, image)
  img_components = image.split('_')
  age_label = int(img_components[0])
  gender_label = int(img_components[1])

  age_labels.append(age_label)
  gender_labels.append(gender_label)
  image_paths.append(image_path)

In [ ]:
gender_mapping = {
    1: 'Female',
    0: 'Male'
}

In [ ]:

df = pd.DataFrame()
df['image_path'], df['age'], df['gender'] = image_paths, age_labels, gender_labels
df.drop(df[df.gender==3].index , inplace=True)
df

In [ ]:
from keras.utils import load_img
from PIL import Image
def load(path):
#   return np.array( [tf.keras.utils.img_to_array(load_img(img,color_mode="grayscale" ).resize((128, 128), Image.Resampling.LANCZOS)) for img in tqdm(path)] )
  return np.array( [tf.keras.utils.img_to_array(load_img(img )) for img in tqdm(path)] )



In [ ]:
sns.set_theme()
sns.displot(df.age,kde=True, bins=30)

In [ ]:
under4s = []

for i in range(len(df)):
    if df['age'].iloc[i] <= 4:
        under4s.append(df.iloc[i])
under4s = pd.DataFrame(under4s)
under4s = under4s.sample(frac=0.3)

df = df[df['age'] > 4]

df = pd.concat([df, under4s], ignore_index = True)

In [ ]:
sns.displot(df['age'],kde=True, bins=30)

In [ ]:
df = df[df['age'] < 80]

In [ ]:
sns.displot(df['age'],kde=True, bins=20)

In [ ]:
# df.gender=df.gender.
plt.hist(df.gender.astype("object").replace(gender_mapping,regex=True),bins=3)
plt.show()

In [ ]:
# X = (load(df["image_path"]).astype('float32')) / 255.
X = (load(df["image_path"]).astype('float32'))
y=df.iloc[:,1:].values.astype("float32")

In [ ]:
from sklearn.model_selection import ShuffleSplit

split_ = ShuffleSplit(n_splits=1, test_size=0.2)
train_index, test_index = next(split_.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
del X
del y

In [ ]:

datagen_train = ImageDataGenerator(
    # preprocessing_function=keras.applications.resnet50.preprocess_input,
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    # vertical_flip=True,
    # featurewise_center=True,
    # featurewise_std_normalization=True,
    rescale=1./255

                             )
# calculate mean on training dataset

datagen_test = ImageDataGenerator(
    # preprocessing_function=keras.applications.resnet50.preprocess_input,
    rescale=1./255
)
# prepare an iterators to scale images
train_age = datagen_train.flow(X_train, y_train[:,0], batch_size=128)
test_age = datagen_test.flow(X_test, y_test[:,0], batch_size=128)

In [ ]:
train_gender = datagen_train.flow(X_train, y_train[:,1], batch_size=32)
test_gender = datagen_test.flow(X_test, y_test[:,1], batch_size=32)

In [ ]:
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model

vgg = vgg16.VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (200, 200, 3))

for layer in vgg.layers:
    layer.trainable = False

def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(1, activation='relu')(top_model)
    return top_model
num_classes = 2

FC_Head = lw(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())

In [ ]:
model.compile(loss = 'mse',
              optimizer = keras.optimizers.Adam(learning_rate = 0.001),
              )


In [ ]:
history2 = model.fit(
    train_age,
    epochs=50,
    shuffle=True,
    validation_data=test_age,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience =4 )]
                        )

In [ ]:

vgg = vgg16.VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (200, 200, 3))

for layer in vgg.layers:
    layer.trainable = False

def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(1)(top_model)
    return top_model
num_classes = 2

FC_Head = lw(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
history2 = model.fit(
    train_gender,
    epochs=50,
    shuffle=True,
    validation_data=test_gender,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience =5 )]
                        )